In [1]:
## This file implements neural networks and logistic regression on p0017SpresabsSTCC_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 84.82% for combination data, and 81.12% for over-sampling data 
## after improvements and regularization.
## To improve accuracy, we use lasso to select important features and thus have new training and test datasets.
## However, with the new datasets, the accuracy becomes 89.19% for combination data and 79.02% for over-sampling data.
## For logistic regression, the accuracies are 96.43% for combination data and 94.41% for over-sampling data.
## For random forest, the accuracy is 96.43% for combination and 95.1% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 98.45% for combination data, and 97.58% for over-sampling.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0017SpresabsSTCC_quant.csv')
df.shape

(255, 148)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0.214333
1      0.350500
2      0.517333
3      0.188833
4      0.191833
5      0.348667
6      0.182667
7      0.207000
8      0.405500
9      0.173500
10     0.184000
11     0.528667
12     0.211667
13     0.218667
14     0.167500
15     0.183333
16     0.406833
17     0.407333
18     0.321167
19     0.177167
20     0.271000
21     0.241833
22     0.178333
23     0.172500
24     0.205833
25     0.170000
26     0.224667
27     0.170500
28     0.163667
29     0.471000
         ...   
225    0.303700
226    0.310600
227    0.443300
228    0.388000
229    0.541200
230    0.378667
231    0.242167
232    0.261500
233    0.295833
234    0.193167
235    0.391333
236    0.160500
237    0.159333
238    0.172000
239    0.262000
240    0.244833
241    0.279833
242    0.200167
243    0.168333
244    0.548667
245    0.164167
246    0.202333
247    0.194167
248    0.188167
249    0.203333
250    0.444667
251    0.238000
252    0.167000
253    0.314333
254    0.252000
Name: pheno, Length: 255

In [6]:
df.head()

,id,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTCCAGTAAT,TTTTAATACATAT,TTTTAAATATTATAA,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,...,group_3904,group_426,group_475,group_6375,group_7822,group_8071,group_8913,pheno,ST,CC
0,107,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0.214333,5,5
1,109,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0.350500,8,8
2,115,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0.517333,5,5
3,120335,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0.188833,5,5
4,120337,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0.191833,5,5


In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

0    237
1     18
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 148)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 147)

In [13]:
df_clean.head()

,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTCCAGTAAT,TTTTAATACATAT,TTTTAAATATTATAA,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,TTCCATCGAATCAC,...,group_3904,group_426,group_475,group_6375,group_7822,group_8071,group_8913,pheno,ST,CC
0,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,5,5
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,8,8
2,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,1,5,5
3,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,5,5
4,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,5,5


In [14]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 146) (255,)


In [15]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 191), (1, 180)]


Using TensorFlow backend.


In [16]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 237), (1, 237)]


In [17]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [18]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [19]:
############# Fully-Connected Neural Network ################

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [21]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [22]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 259 samples, validate on 112 samples
Epoch 1/1000
259/259 [==============================] - 0s 1ms/step - loss: 10.9485 - accuracy: 0.4865 - val_loss: 0.9388 - val_accuracy: 0.5000
Epoch 2/1000
259/259 [==============================] - 0s 217us/step - loss: 0.7826 - accuracy: 0.5637 - val_loss: 0.8678 - val_accuracy: 0.5000
Epoch 3/1000
259/259 [==============================] - 0s 160us/step - loss: 0.7082 - accuracy: 0.5637 - val_loss: 0.6891 - val_accuracy: 0.5893
Epoch 4/1000
259/259 [==============================] - 0s 167us/step - loss: 0.6722 - accuracy: 0.6023 - val_loss: 0.6812 - val_accuracy: 0.5804
Epoch 5/1000
259/259 [==============================] - 0s 339us/step - loss: 0.6647 - accuracy: 0.5907 - val_loss: 0.6738 - val_accuracy: 0.5625
Epoch 6/1000
259/259 [==============================] - 0s 210us/step - loss: 0.6565 - accuracy: 0.5907 - val_loss: 0.6645 - val_accuracy: 0.5268
Epoch 7/1000
259/259 [==============================] - 0s 153us/step - loss: 0

259/259 [==============================] - 0s 106us/step - loss: 0.5061 - accuracy: 0.7452 - val_loss: 0.5287 - val_accuracy: 0.7500
Epoch 57/1000
259/259 [==============================] - 0s 86us/step - loss: 0.5041 - accuracy: 0.7413 - val_loss: 0.5262 - val_accuracy: 0.7589
Epoch 58/1000
259/259 [==============================] - 0s 99us/step - loss: 0.5029 - accuracy: 0.7490 - val_loss: 0.5223 - val_accuracy: 0.7589
Epoch 59/1000
259/259 [==============================] - 0s 101us/step - loss: 0.4994 - accuracy: 0.7413 - val_loss: 0.5206 - val_accuracy: 0.7589
Epoch 60/1000
259/259 [==============================] - 0s 86us/step - loss: 0.4969 - accuracy: 0.7529 - val_loss: 0.5184 - val_accuracy: 0.7589
Epoch 61/1000
259/259 [==============================] - 0s 105us/step - loss: 0.4941 - accuracy: 0.7529 - val_loss: 0.5200 - val_accuracy: 0.7500
Epoch 62/1000
259/259 [==============================] - 0s 100us/step - loss: 0.4960 - accuracy: 0.7529 - val_loss: 0.5239 - val_accur

Epoch 112/1000
259/259 [==============================] - 0s 87us/step - loss: 0.4111 - accuracy: 0.7761 - val_loss: 0.4690 - val_accuracy: 0.7143
Epoch 113/1000
259/259 [==============================] - 0s 154us/step - loss: 0.4189 - accuracy: 0.7799 - val_loss: 0.4477 - val_accuracy: 0.7589
Epoch 114/1000
259/259 [==============================] - 0s 124us/step - loss: 0.4072 - accuracy: 0.7722 - val_loss: 0.4523 - val_accuracy: 0.7589
Epoch 115/1000
259/259 [==============================] - 0s 133us/step - loss: 0.4101 - accuracy: 0.7722 - val_loss: 0.4456 - val_accuracy: 0.7679
Epoch 116/1000
259/259 [==============================] - 0s 127us/step - loss: 0.4058 - accuracy: 0.7645 - val_loss: 0.4464 - val_accuracy: 0.7500
Epoch 117/1000
259/259 [==============================] - 0s 126us/step - loss: 0.4027 - accuracy: 0.7722 - val_loss: 0.4646 - val_accuracy: 0.7143
Epoch 118/1000
259/259 [==============================] - 0s 134us/step - loss: 0.4177 - accuracy: 0.7568 - val_l

259/259 [==============================] - 0s 124us/step - loss: 0.3623 - accuracy: 0.8069 - val_loss: 0.4030 - val_accuracy: 0.8214
Epoch 168/1000
259/259 [==============================] - 0s 130us/step - loss: 0.3570 - accuracy: 0.8108 - val_loss: 0.4022 - val_accuracy: 0.7679
Epoch 169/1000
259/259 [==============================] - 0s 178us/step - loss: 0.3601 - accuracy: 0.8031 - val_loss: 0.3996 - val_accuracy: 0.7946
Epoch 170/1000
259/259 [==============================] - 0s 167us/step - loss: 0.3566 - accuracy: 0.7992 - val_loss: 0.4018 - val_accuracy: 0.8125
Epoch 171/1000
259/259 [==============================] - 0s 145us/step - loss: 0.3564 - accuracy: 0.7954 - val_loss: 0.4005 - val_accuracy: 0.8214
Epoch 172/1000
259/259 [==============================] - 0s 119us/step - loss: 0.3545 - accuracy: 0.8069 - val_loss: 0.4014 - val_accuracy: 0.7946
Epoch 173/1000
259/259 [==============================] - 0s 94us/step - loss: 0.3572 - accuracy: 0.8069 - val_loss: 0.3977 - v

259/259 [==============================] - 0s 89us/step - loss: 0.3180 - accuracy: 0.8263 - val_loss: 0.4110 - val_accuracy: 0.7946
Epoch 223/1000
259/259 [==============================] - 0s 94us/step - loss: 0.3311 - accuracy: 0.8108 - val_loss: 0.3624 - val_accuracy: 0.8214
Epoch 224/1000
259/259 [==============================] - 0s 92us/step - loss: 0.3195 - accuracy: 0.8340 - val_loss: 0.3589 - val_accuracy: 0.8125
Epoch 225/1000
259/259 [==============================] - 0s 91us/step - loss: 0.3233 - accuracy: 0.8224 - val_loss: 0.3718 - val_accuracy: 0.8036
Epoch 226/1000
259/259 [==============================] - 0s 124us/step - loss: 0.3157 - accuracy: 0.8263 - val_loss: 0.3575 - val_accuracy: 0.8304
Epoch 227/1000
259/259 [==============================] - 0s 285us/step - loss: 0.3136 - accuracy: 0.8340 - val_loss: 0.3569 - val_accuracy: 0.8125
Epoch 228/1000
259/259 [==============================] - 0s 99us/step - loss: 0.3157 - accuracy: 0.8185 - val_loss: 0.3556 - val_a

...

259/259 [==============================] - 0s 126us/step - loss: 0.2429 - accuracy: 0.8494 - val_loss: 0.3225 - val_accuracy: 0.8304
Epoch 775/1000
259/259 [==============================] - 0s 93us/step - loss: 0.2470 - accuracy: 0.8494 - val_loss: 0.3197 - val_accuracy: 0.8304
Epoch 776/1000
259/259 [==============================] - 0s 82us/step - loss: 0.2457 - accuracy: 0.8494 - val_loss: 0.3203 - val_accuracy: 0.8304
Epoch 777/1000
259/259 [==============================] - 0s 93us/step - loss: 0.2413 - accuracy: 0.8533 - val_loss: 0.3182 - val_accuracy: 0.8304
Epoch 778/1000
259/259 [==============================] - 0s 101us/step - loss: 0.2415 - accuracy: 0.8494 - val_loss: 0.3203 - val_accuracy: 0.8304
Epoch 779/1000
259/259 [==============================] - 0s 150us/step - loss: 0.2429 - accuracy: 0.8494 - val_loss: 0.3137 - val_accuracy: 0.8393
Epoch 780/1000
259/259 [==============================] - 0s 168us/step - loss: 0.2456 - accuracy: 0.8417 - val_loss: 0.3115 - val

259/259 [==============================] - 0s 80us/step - loss: 0.2373 - accuracy: 0.8571 - val_loss: 0.3142 - val_accuracy: 0.8482
Epoch 830/1000
259/259 [==============================] - 0s 83us/step - loss: 0.2423 - accuracy: 0.8494 - val_loss: 0.3248 - val_accuracy: 0.8304
Epoch 831/1000
259/259 [==============================] - 0s 83us/step - loss: 0.2389 - accuracy: 0.8533 - val_loss: 0.3161 - val_accuracy: 0.8482
Epoch 832/1000
259/259 [==============================] - 0s 87us/step - loss: 0.2372 - accuracy: 0.8610 - val_loss: 0.3204 - val_accuracy: 0.8304
Epoch 833/1000
259/259 [==============================] - 0s 93us/step - loss: 0.2393 - accuracy: 0.8494 - val_loss: 0.3145 - val_accuracy: 0.8482
Epoch 834/1000
259/259 [==============================] - 0s 86us/step - loss: 0.2366 - accuracy: 0.8571 - val_loss: 0.3162 - val_accuracy: 0.8393
Epoch 835/1000
259/259 [==============================] - 0s 89us/step - loss: 0.2376 - accuracy: 0.8571 - val_loss: 0.3230 - val_acc

259/259 [==============================] - 0s 110us/step - loss: 0.2338 - accuracy: 0.8610 - val_loss: 0.3135 - val_accuracy: 0.8482
Epoch 885/1000
259/259 [==============================] - 0s 102us/step - loss: 0.2324 - accuracy: 0.8610 - val_loss: 0.3138 - val_accuracy: 0.8482
Epoch 886/1000
259/259 [==============================] - 0s 129us/step - loss: 0.2333 - accuracy: 0.8610 - val_loss: 0.3118 - val_accuracy: 0.8482
Epoch 887/1000
259/259 [==============================] - 0s 103us/step - loss: 0.2328 - accuracy: 0.8610 - val_loss: 0.3073 - val_accuracy: 0.8482
Epoch 888/1000
259/259 [==============================] - 0s 145us/step - loss: 0.2328 - accuracy: 0.8610 - val_loss: 0.3093 - val_accuracy: 0.8482
Epoch 889/1000
259/259 [==============================] - 0s 214us/step - loss: 0.2326 - accuracy: 0.8533 - val_loss: 0.3127 - val_accuracy: 0.8482
Epoch 890/1000
259/259 [==============================] - 0s 199us/step - loss: 0.2321 - accuracy: 0.8610 - val_loss: 0.3077 - 

259/259 [==============================] - 0s 92us/step - loss: 0.2316 - accuracy: 0.8610 - val_loss: 0.3013 - val_accuracy: 0.8482
Epoch 940/1000
259/259 [==============================] - 0s 84us/step - loss: 0.2318 - accuracy: 0.8610 - val_loss: 0.3008 - val_accuracy: 0.8482
Epoch 941/1000
259/259 [==============================] - 0s 83us/step - loss: 0.2319 - accuracy: 0.8610 - val_loss: 0.3026 - val_accuracy: 0.8482
Epoch 942/1000
259/259 [==============================] - 0s 92us/step - loss: 0.2307 - accuracy: 0.8610 - val_loss: 0.3059 - val_accuracy: 0.8482
Epoch 943/1000
259/259 [==============================] - 0s 81us/step - loss: 0.2318 - accuracy: 0.8610 - val_loss: 0.3058 - val_accuracy: 0.8482
Epoch 944/1000
259/259 [==============================] - 0s 102us/step - loss: 0.2319 - accuracy: 0.8610 - val_loss: 0.3126 - val_accuracy: 0.8482
Epoch 945/1000
259/259 [==============================] - 0s 75us/step - loss: 0.2320 - accuracy: 0.8610 - val_loss: 0.3087 - val_ac

259/259 [==============================] - 0s 98us/step - loss: 0.2296 - accuracy: 0.8610 - val_loss: 0.3091 - val_accuracy: 0.8482
Epoch 995/1000
259/259 [==============================] - 0s 89us/step - loss: 0.2286 - accuracy: 0.8610 - val_loss: 0.3075 - val_accuracy: 0.8482
Epoch 996/1000
259/259 [==============================] - 0s 96us/step - loss: 0.2272 - accuracy: 0.8610 - val_loss: 0.3062 - val_accuracy: 0.8482
Epoch 997/1000
259/259 [==============================] - 0s 75us/step - loss: 0.2283 - accuracy: 0.8610 - val_loss: 0.3043 - val_accuracy: 0.8482
Epoch 998/1000
259/259 [==============================] - 0s 78us/step - loss: 0.2293 - accuracy: 0.8610 - val_loss: 0.3059 - val_accuracy: 0.8393
Epoch 999/1000
259/259 [==============================] - 0s 72us/step - loss: 0.2306 - accuracy: 0.8610 - val_loss: 0.3056 - val_accuracy: 0.8482
Epoch 1000/1000
259/259 [==============================] - 0s 95us/step - loss: 0.2298 - accuracy: 0.8610 - val_loss: 0.3093 - val_ac

In [24]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

112/112 [==============================] - 0s 60us/step
combination test accuracy: 84.82%


In [82]:
#### improve neural network
model1_comb3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [83]:
model1_comb3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [84]:
model1_comb3.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 259 samples, validate on 112 samples
Epoch 1/1000
259/259 [==============================] - 0s 442us/step - loss: 2.2571 - accuracy: 0.5097 - val_loss: 3.4530 - val_accuracy: 0.4732
Epoch 2/1000
259/259 [==============================] - 0s 84us/step - loss: 1.4424 - accuracy: 0.6486 - val_loss: 0.9518 - val_accuracy: 0.5179
Epoch 3/1000
259/259 [==============================] - 0s 103us/step - loss: 0.7185 - accuracy: 0.6564 - val_loss: 0.7480 - val_accuracy: 0.6429
Epoch 4/1000
259/259 [==============================] - 0s 93us/step - loss: 0.6758 - accuracy: 0.6332 - val_loss: 0.6531 - val_accuracy: 0.5893
Epoch 5/1000
259/259 [==============================] - 0s 94us/step - loss: 0.6439 - accuracy: 0.6718 - val_loss: 0.6524 - val_accuracy: 0.5982
Epoch 6/1000
259/259 [==============================] - 0s 90us/step - loss: 0.6443 - accuracy: 0.6795 - val_loss: 0.6822 - val_accuracy: 0.6429
Epoch 7/1000
259/259 [==============================] - 0s 100us/step - loss: 0.66

259/259 [==============================] - 0s 65us/step - loss: 0.4825 - accuracy: 0.7143 - val_loss: 0.6009 - val_accuracy: 0.7054
Epoch 57/1000
259/259 [==============================] - 0s 73us/step - loss: 0.4976 - accuracy: 0.7413 - val_loss: 0.4993 - val_accuracy: 0.7500
Epoch 58/1000
259/259 [==============================] - 0s 63us/step - loss: 0.4655 - accuracy: 0.7452 - val_loss: 0.5000 - val_accuracy: 0.7321
Epoch 59/1000
259/259 [==============================] - 0s 64us/step - loss: 0.4597 - accuracy: 0.7413 - val_loss: 0.4938 - val_accuracy: 0.7411
Epoch 60/1000
259/259 [==============================] - 0s 64us/step - loss: 0.4559 - accuracy: 0.7413 - val_loss: 0.5332 - val_accuracy: 0.7143
Epoch 61/1000
259/259 [==============================] - 0s 64us/step - loss: 0.4708 - accuracy: 0.7375 - val_loss: 0.5380 - val_accuracy: 0.7232
Epoch 62/1000
259/259 [==============================] - 0s 86us/step - loss: 0.4778 - accuracy: 0.7181 - val_loss: 0.4904 - val_accuracy:

259/259 [==============================] - 0s 90us/step - loss: 0.3939 - accuracy: 0.7838 - val_loss: 0.4277 - val_accuracy: 0.7589
Epoch 113/1000
259/259 [==============================] - 0s 96us/step - loss: 0.3696 - accuracy: 0.7799 - val_loss: 0.4267 - val_accuracy: 0.7946
Epoch 114/1000
259/259 [==============================] - 0s 83us/step - loss: 0.3742 - accuracy: 0.7915 - val_loss: 0.4257 - val_accuracy: 0.7946
Epoch 115/1000
259/259 [==============================] - 0s 74us/step - loss: 0.3729 - accuracy: 0.8031 - val_loss: 0.4448 - val_accuracy: 0.7500
Epoch 116/1000
259/259 [==============================] - 0s 74us/step - loss: 0.3802 - accuracy: 0.7915 - val_loss: 0.4139 - val_accuracy: 0.8125
Epoch 117/1000
259/259 [==============================] - 0s 86us/step - loss: 0.3614 - accuracy: 0.7992 - val_loss: 0.4400 - val_accuracy: 0.7946
Epoch 118/1000
259/259 [==============================] - 0s 88us/step - loss: 0.3613 - accuracy: 0.7954 - val_loss: 0.4097 - val_acc

Epoch 168/1000
259/259 [==============================] - 0s 97us/step - loss: 0.2960 - accuracy: 0.8610 - val_loss: 0.3524 - val_accuracy: 0.8304
Epoch 169/1000
259/259 [==============================] - 0s 77us/step - loss: 0.2892 - accuracy: 0.8610 - val_loss: 0.3668 - val_accuracy: 0.8125
Epoch 170/1000
259/259 [==============================] - 0s 83us/step - loss: 0.3048 - accuracy: 0.8417 - val_loss: 0.5625 - val_accuracy: 0.7411
Epoch 171/1000
259/259 [==============================] - 0s 91us/step - loss: 0.3457 - accuracy: 0.8185 - val_loss: 0.3489 - val_accuracy: 0.8482
Epoch 172/1000
259/259 [==============================] - 0s 91us/step - loss: 0.2923 - accuracy: 0.8494 - val_loss: 0.3603 - val_accuracy: 0.8393
Epoch 173/1000
259/259 [==============================] - 0s 77us/step - loss: 0.2944 - accuracy: 0.8533 - val_loss: 0.3458 - val_accuracy: 0.8393
Epoch 174/1000
259/259 [==============================] - 0s 81us/step - loss: 0.2942 - accuracy: 0.8571 - val_loss: 0

Epoch 224/1000
259/259 [==============================] - 0s 76us/step - loss: 0.2428 - accuracy: 0.8996 - val_loss: 0.3039 - val_accuracy: 0.9107
Epoch 225/1000
259/259 [==============================] - 0s 66us/step - loss: 0.2821 - accuracy: 0.8378 - val_loss: 0.3258 - val_accuracy: 0.8214
Epoch 226/1000
259/259 [==============================] - 0s 62us/step - loss: 0.2678 - accuracy: 0.8571 - val_loss: 0.3764 - val_accuracy: 0.7321
Epoch 227/1000
259/259 [==============================] - 0s 63us/step - loss: 0.3169 - accuracy: 0.7954 - val_loss: 0.3698 - val_accuracy: 0.8214
Epoch 228/1000
259/259 [==============================] - 0s 72us/step - loss: 0.2924 - accuracy: 0.8456 - val_loss: 0.3288 - val_accuracy: 0.8393
Epoch 229/1000
259/259 [==============================] - 0s 76us/step - loss: 0.2590 - accuracy: 0.8571 - val_loss: 0.3409 - val_accuracy: 0.8304
Epoch 230/1000
259/259 [==============================] - 0s 80us/step - loss: 0.2580 - accuracy: 0.8456 - val_loss: 0

...

259/259 [==============================] - 0s 47us/step - loss: 0.2522 - accuracy: 0.8378 - val_loss: 0.3926 - val_accuracy: 0.8393
Epoch 725/1000
259/259 [==============================] - 0s 54us/step - loss: 0.2497 - accuracy: 0.8378 - val_loss: 0.3927 - val_accuracy: 0.8393
Epoch 726/1000
259/259 [==============================] - 0s 49us/step - loss: 0.2459 - accuracy: 0.8417 - val_loss: 0.3965 - val_accuracy: 0.8393
Epoch 727/1000
259/259 [==============================] - 0s 48us/step - loss: 0.2522 - accuracy: 0.8378 - val_loss: 0.3909 - val_accuracy: 0.8393
Epoch 728/1000
259/259 [==============================] - 0s 47us/step - loss: 0.2430 - accuracy: 0.8456 - val_loss: 0.3895 - val_accuracy: 0.8393
Epoch 729/1000
259/259 [==============================] - 0s 49us/step - loss: 0.2467 - accuracy: 0.8417 - val_loss: 0.3901 - val_accuracy: 0.8393
Epoch 730/1000
259/259 [==============================] - 0s 51us/step - loss: 0.2499 - accuracy: 0.8456 - val_loss: 0.3917 - val_acc

Epoch 780/1000
259/259 [==============================] - 0s 78us/step - loss: 0.2196 - accuracy: 0.8726 - val_loss: 0.4051 - val_accuracy: 0.8304
Epoch 781/1000
259/259 [==============================] - 0s 78us/step - loss: 0.2225 - accuracy: 0.8687 - val_loss: 0.3947 - val_accuracy: 0.8393
Epoch 782/1000
259/259 [==============================] - 0s 88us/step - loss: 0.2178 - accuracy: 0.8649 - val_loss: 0.3773 - val_accuracy: 0.8482
Epoch 783/1000
259/259 [==============================] - 0s 117us/step - loss: 0.2172 - accuracy: 0.8726 - val_loss: 0.3878 - val_accuracy: 0.8393
Epoch 784/1000
259/259 [==============================] - 0s 168us/step - loss: 0.2170 - accuracy: 0.8726 - val_loss: 0.3860 - val_accuracy: 0.8393
Epoch 785/1000
259/259 [==============================] - 0s 82us/step - loss: 0.2132 - accuracy: 0.8726 - val_loss: 0.3620 - val_accuracy: 0.8571
Epoch 786/1000
259/259 [==============================] - 0s 132us/step - loss: 0.2022 - accuracy: 0.8687 - val_loss

259/259 [==============================] - 0s 140us/step - loss: 0.2770 - accuracy: 0.8301 - val_loss: 0.4129 - val_accuracy: 0.7946
Epoch 836/1000
259/259 [==============================] - 0s 133us/step - loss: 0.2785 - accuracy: 0.8263 - val_loss: 0.4129 - val_accuracy: 0.7857
Epoch 837/1000
259/259 [==============================] - 0s 124us/step - loss: 0.2785 - accuracy: 0.8263 - val_loss: 0.4133 - val_accuracy: 0.7857
Epoch 838/1000
259/259 [==============================] - 0s 157us/step - loss: 0.2779 - accuracy: 0.8263 - val_loss: 0.4140 - val_accuracy: 0.7857
Epoch 839/1000
259/259 [==============================] - 0s 138us/step - loss: 0.2776 - accuracy: 0.8263 - val_loss: 0.4187 - val_accuracy: 0.7857
Epoch 840/1000
259/259 [==============================] - 0s 117us/step - loss: 0.2773 - accuracy: 0.8301 - val_loss: 0.4155 - val_accuracy: 0.7946
Epoch 841/1000
259/259 [==============================] - 0s 86us/step - loss: 0.2756 - accuracy: 0.8263 - val_loss: 0.4138 - v

Epoch 891/1000
259/259 [==============================] - 0s 94us/step - loss: 0.2950 - accuracy: 0.7722 - val_loss: 0.4435 - val_accuracy: 0.7589
Epoch 892/1000
259/259 [==============================] - 0s 89us/step - loss: 0.2876 - accuracy: 0.7722 - val_loss: 0.4374 - val_accuracy: 0.7589
Epoch 893/1000
259/259 [==============================] - 0s 84us/step - loss: 0.2852 - accuracy: 0.7722 - val_loss: 0.4362 - val_accuracy: 0.7589
Epoch 894/1000
259/259 [==============================] - 0s 83us/step - loss: 0.2842 - accuracy: 0.7722 - val_loss: 0.4378 - val_accuracy: 0.7589
Epoch 895/1000
259/259 [==============================] - 0s 86us/step - loss: 0.2841 - accuracy: 0.7722 - val_loss: 0.4364 - val_accuracy: 0.7589
Epoch 896/1000
259/259 [==============================] - 0s 80us/step - loss: 0.2823 - accuracy: 0.7915 - val_loss: 0.4249 - val_accuracy: 0.7768
Epoch 897/1000
259/259 [==============================] - 0s 80us/step - loss: 0.2573 - accuracy: 0.8301 - val_loss: 0

Epoch 947/1000
259/259 [==============================] - 0s 51us/step - loss: 0.2782 - accuracy: 0.8340 - val_loss: 0.4320 - val_accuracy: 0.7857
Epoch 948/1000
259/259 [==============================] - 0s 53us/step - loss: 0.2787 - accuracy: 0.8340 - val_loss: 0.4322 - val_accuracy: 0.7857
Epoch 949/1000
259/259 [==============================] - 0s 52us/step - loss: 0.2776 - accuracy: 0.8340 - val_loss: 0.4353 - val_accuracy: 0.7768
Epoch 950/1000
259/259 [==============================] - 0s 63us/step - loss: 0.3047 - accuracy: 0.8108 - val_loss: 0.4413 - val_accuracy: 0.7768
Epoch 951/1000
259/259 [==============================] - 0s 69us/step - loss: 0.2858 - accuracy: 0.8263 - val_loss: 0.4434 - val_accuracy: 0.7768
Epoch 952/1000
259/259 [==============================] - 0s 73us/step - loss: 0.2842 - accuracy: 0.8263 - val_loss: 0.4446 - val_accuracy: 0.7768
Epoch 953/1000
259/259 [==============================] - 0s 74us/step - loss: 0.2835 - accuracy: 0.8301 - val_loss: 0

In [85]:
acc_test_comb3 = model1_comb3.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

112/112 [==============================] - 0s 134us/step
combination test accuracy: 77.68%


In [29]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [30]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 349us/step - loss: 5.4500 - accuracy: 0.4955 - val_loss: 0.6906 - val_accuracy: 0.6154
Epoch 2/1000
331/331 [==============================] - 0s 68us/step - loss: 0.7022 - accuracy: 0.5801 - val_loss: 0.6776 - val_accuracy: 0.6713
Epoch 3/1000
331/331 [==============================] - 0s 81us/step - loss: 0.6925 - accuracy: 0.6314 - val_loss: 0.6693 - val_accuracy: 0.6643
Epoch 4/1000
331/331 [==============================] - 0s 72us/step - loss: 0.6850 - accuracy: 0.6042 - val_loss: 0.6617 - val_accuracy: 0.6923
Epoch 5/1000
331/331 [==============================] - 0s 75us/step - loss: 0.6766 - accuracy: 0.6012 - val_loss: 0.6559 - val_accuracy: 0.6923
Epoch 6/1000
331/331 [==============================] - 0s 75us/step - loss: 0.6706 - accuracy: 0.6073 - val_loss: 0.6517 - val_accuracy: 0.6573
Epoch 7/1000
331/331 [==============================] - 0s 72us/step - loss: 0.6695

Epoch 57/1000
331/331 [==============================] - 0s 73us/step - loss: 0.5540 - accuracy: 0.7372 - val_loss: 0.5356 - val_accuracy: 0.7203
Epoch 58/1000
331/331 [==============================] - 0s 64us/step - loss: 0.5503 - accuracy: 0.7341 - val_loss: 0.5320 - val_accuracy: 0.7413
Epoch 59/1000
331/331 [==============================] - 0s 69us/step - loss: 0.5501 - accuracy: 0.7492 - val_loss: 0.5307 - val_accuracy: 0.7343
Epoch 60/1000
331/331 [==============================] - 0s 70us/step - loss: 0.5469 - accuracy: 0.7462 - val_loss: 0.5331 - val_accuracy: 0.7203
Epoch 61/1000
331/331 [==============================] - 0s 70us/step - loss: 0.5485 - accuracy: 0.7402 - val_loss: 0.5267 - val_accuracy: 0.7413
Epoch 62/1000
331/331 [==============================] - 0s 70us/step - loss: 0.5437 - accuracy: 0.7432 - val_loss: 0.5578 - val_accuracy: 0.6783
Epoch 63/1000
331/331 [==============================] - 0s 76us/step - loss: 0.5495 - accuracy: 0.7190 - val_loss: 0.5407 -

Epoch 113/1000
331/331 [==============================] - 0s 69us/step - loss: 0.4844 - accuracy: 0.7613 - val_loss: 0.4667 - val_accuracy: 0.7902
Epoch 114/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4825 - accuracy: 0.7523 - val_loss: 0.4668 - val_accuracy: 0.7203
Epoch 115/1000
331/331 [==============================] - 0s 70us/step - loss: 0.4825 - accuracy: 0.7553 - val_loss: 0.5010 - val_accuracy: 0.7203
Epoch 116/1000
331/331 [==============================] - 0s 63us/step - loss: 0.4853 - accuracy: 0.7613 - val_loss: 0.4705 - val_accuracy: 0.7203
Epoch 117/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4827 - accuracy: 0.7523 - val_loss: 0.4655 - val_accuracy: 0.7483
Epoch 118/1000
331/331 [==============================] - 0s 69us/step - loss: 0.4803 - accuracy: 0.7583 - val_loss: 0.4543 - val_accuracy: 0.7692
Epoch 119/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4799 - accuracy: 0.7644 - val_loss: 0

331/331 [==============================] - 0s 63us/step - loss: 0.4439 - accuracy: 0.7644 - val_loss: 0.4161 - val_accuracy: 0.7692
Epoch 169/1000
331/331 [==============================] - 0s 63us/step - loss: 0.4398 - accuracy: 0.7613 - val_loss: 0.4247 - val_accuracy: 0.7692
Epoch 170/1000
331/331 [==============================] - 0s 61us/step - loss: 0.4391 - accuracy: 0.7613 - val_loss: 0.4141 - val_accuracy: 0.7972
Epoch 171/1000
331/331 [==============================] - 0s 63us/step - loss: 0.4419 - accuracy: 0.7674 - val_loss: 0.4279 - val_accuracy: 0.7692
Epoch 172/1000
331/331 [==============================] - 0s 63us/step - loss: 0.4399 - accuracy: 0.7704 - val_loss: 0.4113 - val_accuracy: 0.7692
Epoch 173/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4332 - accuracy: 0.7734 - val_loss: 0.4153 - val_accuracy: 0.7692
Epoch 174/1000
331/331 [==============================] - 0s 63us/step - loss: 0.4423 - accuracy: 0.7583 - val_loss: 0.4114 - val_acc

331/331 [==============================] - 0s 121us/step - loss: 0.4082 - accuracy: 0.7704 - val_loss: 0.3928 - val_accuracy: 0.7692
Epoch 224/1000
331/331 [==============================] - 0s 541us/step - loss: 0.4052 - accuracy: 0.7795 - val_loss: 0.3738 - val_accuracy: 0.8322
Epoch 225/1000
331/331 [==============================] - 0s 149us/step - loss: 0.4092 - accuracy: 0.7825 - val_loss: 0.3736 - val_accuracy: 0.7692
Epoch 226/1000
331/331 [==============================] - 0s 113us/step - loss: 0.4125 - accuracy: 0.7764 - val_loss: 0.3720 - val_accuracy: 0.8042
Epoch 227/1000
331/331 [==============================] - 0s 113us/step - loss: 0.4206 - accuracy: 0.7613 - val_loss: 0.3982 - val_accuracy: 0.7832
Epoch 228/1000
331/331 [==============================] - 0s 105us/step - loss: 0.4029 - accuracy: 0.7764 - val_loss: 0.3785 - val_accuracy: 0.7692
Epoch 229/1000
331/331 [==============================] - 0s 98us/step - loss: 0.4053 - accuracy: 0.7734 - val_loss: 0.3949 - v

...

Epoch 726/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3331 - accuracy: 0.8066 - val_loss: 0.2792 - val_accuracy: 0.8322
Epoch 727/1000
331/331 [==============================] - 0s 77us/step - loss: 0.3694 - accuracy: 0.8006 - val_loss: 0.2828 - val_accuracy: 0.8252
Epoch 728/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3142 - accuracy: 0.8097 - val_loss: 0.2907 - val_accuracy: 0.8182
Epoch 729/1000
331/331 [==============================] - 0s 66us/step - loss: 0.3056 - accuracy: 0.8218 - val_loss: 0.2908 - val_accuracy: 0.8182
Epoch 730/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3703 - accuracy: 0.8006 - val_loss: 0.2800 - val_accuracy: 0.8322
Epoch 731/1000
331/331 [==============================] - 0s 66us/step - loss: 0.3433 - accuracy: 0.7946 - val_loss: 0.2937 - val_accuracy: 0.8042
Epoch 732/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3328 - accuracy: 0.8066 - val_loss: 0

Epoch 782/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3467 - accuracy: 0.8097 - val_loss: 0.2912 - val_accuracy: 0.8182
Epoch 783/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3145 - accuracy: 0.8036 - val_loss: 0.2743 - val_accuracy: 0.8322
Epoch 784/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3020 - accuracy: 0.8218 - val_loss: 0.2794 - val_accuracy: 0.8252
Epoch 785/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3065 - accuracy: 0.8127 - val_loss: 0.2838 - val_accuracy: 0.8392
Epoch 786/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3014 - accuracy: 0.8187 - val_loss: 0.2761 - val_accuracy: 0.8392
Epoch 787/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3095 - accuracy: 0.8066 - val_loss: 0.2856 - val_accuracy: 0.8392
Epoch 788/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3349 - accuracy: 0.8066 - val_loss: 0

331/331 [==============================] - 0s 67us/step - loss: 0.3110 - accuracy: 0.8097 - val_loss: 0.2879 - val_accuracy: 0.8042
Epoch 838/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3345 - accuracy: 0.8066 - val_loss: 0.2729 - val_accuracy: 0.8322
Epoch 839/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3005 - accuracy: 0.8218 - val_loss: 0.2817 - val_accuracy: 0.8392
Epoch 840/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3253 - accuracy: 0.8097 - val_loss: 0.3647 - val_accuracy: 0.8042
Epoch 841/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3068 - accuracy: 0.8187 - val_loss: 0.2853 - val_accuracy: 0.8042
Epoch 842/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3127 - accuracy: 0.8066 - val_loss: 0.2794 - val_accuracy: 0.8252
Epoch 843/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3181 - accuracy: 0.8127 - val_loss: 0.2751 - val_acc

Epoch 893/1000
331/331 [==============================] - 0s 67us/step - loss: 0.2977 - accuracy: 0.8187 - val_loss: 0.2768 - val_accuracy: 0.8252
Epoch 894/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3029 - accuracy: 0.8157 - val_loss: 0.2737 - val_accuracy: 0.8392
Epoch 895/1000
331/331 [==============================] - 0s 69us/step - loss: 0.2961 - accuracy: 0.8218 - val_loss: 0.2706 - val_accuracy: 0.8322
Epoch 896/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3035 - accuracy: 0.8097 - val_loss: 0.2867 - val_accuracy: 0.8042
Epoch 897/1000
331/331 [==============================] - 0s 66us/step - loss: 0.3243 - accuracy: 0.8006 - val_loss: 0.2772 - val_accuracy: 0.8252
Epoch 898/1000
331/331 [==============================] - 0s 64us/step - loss: 0.2964 - accuracy: 0.8157 - val_loss: 0.2714 - val_accuracy: 0.8392
Epoch 899/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3137 - accuracy: 0.8066 - val_loss: 0

Epoch 949/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3039 - accuracy: 0.8097 - val_loss: 0.2720 - val_accuracy: 0.8392
Epoch 950/1000
331/331 [==============================] - 0s 66us/step - loss: 0.3013 - accuracy: 0.8097 - val_loss: 0.2699 - val_accuracy: 0.8392
Epoch 951/1000
331/331 [==============================] - 0s 65us/step - loss: 0.2944 - accuracy: 0.8218 - val_loss: 0.2788 - val_accuracy: 0.8392
Epoch 952/1000
331/331 [==============================] - 0s 62us/step - loss: 0.3161 - accuracy: 0.8127 - val_loss: 0.3401 - val_accuracy: 0.8042
Epoch 953/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3017 - accuracy: 0.8157 - val_loss: 0.2691 - val_accuracy: 0.8392
Epoch 954/1000
331/331 [==============================] - 0s 72us/step - loss: 0.2955 - accuracy: 0.8248 - val_loss: 0.2710 - val_accuracy: 0.8322
Epoch 955/1000
331/331 [==============================] - 0s 67us/step - loss: 0.2919 - accuracy: 0.8248 - val_loss: 0

In [32]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

143/143 [==============================] - 0s 63us/step
over-sampling test accuracy: 80.42%


In [33]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [34]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 353us/step - loss: 11.3501 - accuracy: 0.5378 - val_loss: 0.6912 - val_accuracy: 0.5385
Epoch 2/1000
331/331 [==============================] - 0s 68us/step - loss: 0.6858 - accuracy: 0.5801 - val_loss: 0.7021 - val_accuracy: 0.6014
Epoch 3/1000
331/331 [==============================] - 0s 67us/step - loss: 0.6974 - accuracy: 0.6254 - val_loss: 0.6792 - val_accuracy: 0.6364
Epoch 4/1000
331/331 [==============================] - 0s 76us/step - loss: 0.6759 - accuracy: 0.6103 - val_loss: 0.6705 - val_accuracy: 0.6643
Epoch 5/1000
331/331 [==============================] - 0s 77us/step - loss: 0.6680 - accuracy: 0.6465 - val_loss: 0.6627 - val_accuracy: 0.6923
Epoch 6/1000
331/331 [==============================] - 0s 77us/step - loss: 0.6612 - accuracy: 0.6647 - val_loss: 0.6576 - val_accuracy: 0.7063
Epoch 7/1000
331/331 [==============================] - 0s 77us/step - loss: 0.657

Epoch 57/1000
331/331 [==============================] - 0s 71us/step - loss: 0.5253 - accuracy: 0.7281 - val_loss: 0.5056 - val_accuracy: 0.7133
Epoch 58/1000
331/331 [==============================] - 0s 77us/step - loss: 0.5201 - accuracy: 0.7311 - val_loss: 0.5015 - val_accuracy: 0.7063
Epoch 59/1000
331/331 [==============================] - 0s 68us/step - loss: 0.5174 - accuracy: 0.7251 - val_loss: 0.5051 - val_accuracy: 0.7133
Epoch 60/1000
331/331 [==============================] - 0s 70us/step - loss: 0.5183 - accuracy: 0.7372 - val_loss: 0.5002 - val_accuracy: 0.6993
Epoch 61/1000
331/331 [==============================] - 0s 77us/step - loss: 0.5154 - accuracy: 0.7281 - val_loss: 0.4987 - val_accuracy: 0.7203
Epoch 62/1000
331/331 [==============================] - 0s 70us/step - loss: 0.5113 - accuracy: 0.7311 - val_loss: 0.5018 - val_accuracy: 0.7203
Epoch 63/1000
331/331 [==============================] - 0s 72us/step - loss: 0.5074 - accuracy: 0.7311 - val_loss: 0.4928 -

331/331 [==============================] - 0s 67us/step - loss: 0.4447 - accuracy: 0.7492 - val_loss: 0.4405 - val_accuracy: 0.7552
Epoch 114/1000
331/331 [==============================] - 0s 68us/step - loss: 0.4461 - accuracy: 0.7583 - val_loss: 0.4363 - val_accuracy: 0.7413
Epoch 115/1000
331/331 [==============================] - 0s 69us/step - loss: 0.4436 - accuracy: 0.7492 - val_loss: 0.4306 - val_accuracy: 0.7622
Epoch 116/1000
331/331 [==============================] - 0s 70us/step - loss: 0.4411 - accuracy: 0.7553 - val_loss: 0.4316 - val_accuracy: 0.7483
Epoch 117/1000
331/331 [==============================] - 0s 71us/step - loss: 0.4410 - accuracy: 0.7523 - val_loss: 0.4294 - val_accuracy: 0.7552
Epoch 118/1000
331/331 [==============================] - 0s 73us/step - loss: 0.4600 - accuracy: 0.7372 - val_loss: 0.4378 - val_accuracy: 0.7622
Epoch 119/1000
331/331 [==============================] - 0s 72us/step - loss: 0.4427 - accuracy: 0.7583 - val_loss: 0.4387 - val_acc

Epoch 169/1000
331/331 [==============================] - 0s 69us/step - loss: 0.4160 - accuracy: 0.7795 - val_loss: 0.3919 - val_accuracy: 0.8042
Epoch 170/1000
331/331 [==============================] - 0s 68us/step - loss: 0.4033 - accuracy: 0.8006 - val_loss: 0.3896 - val_accuracy: 0.8182
Epoch 171/1000
331/331 [==============================] - 0s 71us/step - loss: 0.4028 - accuracy: 0.7915 - val_loss: 0.3817 - val_accuracy: 0.8322
Epoch 172/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4006 - accuracy: 0.8006 - val_loss: 0.3914 - val_accuracy: 0.8042
Epoch 173/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3996 - accuracy: 0.8036 - val_loss: 0.3867 - val_accuracy: 0.7762
Epoch 174/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3997 - accuracy: 0.7915 - val_loss: 0.3837 - val_accuracy: 0.7762
Epoch 175/1000
331/331 [==============================] - 0s 76us/step - loss: 0.3989 - accuracy: 0.7976 - val_loss: 0

Epoch 225/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3618 - accuracy: 0.8127 - val_loss: 0.3604 - val_accuracy: 0.8042
Epoch 226/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3678 - accuracy: 0.7946 - val_loss: 0.3445 - val_accuracy: 0.7972
Epoch 227/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3737 - accuracy: 0.7976 - val_loss: 0.3469 - val_accuracy: 0.8042
Epoch 228/1000
331/331 [==============================] - 0s 64us/step - loss: 0.3589 - accuracy: 0.8127 - val_loss: 0.3837 - val_accuracy: 0.7832
Epoch 229/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3772 - accuracy: 0.7855 - val_loss: 0.3769 - val_accuracy: 0.7972
Epoch 230/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3614 - accuracy: 0.8006 - val_loss: 0.3679 - val_accuracy: 0.7832
Epoch 231/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3690 - accuracy: 0.7915 - val_loss: 0

...

Epoch 728/1000
331/331 [==============================] - 0s 72us/step - loss: 0.2861 - accuracy: 0.8278 - val_loss: 0.2786 - val_accuracy: 0.8392
Epoch 729/1000
331/331 [==============================] - 0s 67us/step - loss: 0.2910 - accuracy: 0.8248 - val_loss: 0.3068 - val_accuracy: 0.8252
Epoch 730/1000
331/331 [==============================] - 0s 69us/step - loss: 0.2926 - accuracy: 0.8187 - val_loss: 0.2788 - val_accuracy: 0.8252
Epoch 731/1000
331/331 [==============================] - 0s 73us/step - loss: 0.2906 - accuracy: 0.8157 - val_loss: 0.2959 - val_accuracy: 0.8252
Epoch 732/1000
331/331 [==============================] - 0s 70us/step - loss: 0.2924 - accuracy: 0.8187 - val_loss: 0.2793 - val_accuracy: 0.8392
Epoch 733/1000
331/331 [==============================] - 0s 67us/step - loss: 0.2871 - accuracy: 0.8248 - val_loss: 0.2757 - val_accuracy: 0.8322
Epoch 734/1000
331/331 [==============================] - 0s 77us/step - loss: 0.3030 - accuracy: 0.8097 - val_loss: 0

Epoch 784/1000
331/331 [==============================] - 0s 78us/step - loss: 0.3135 - accuracy: 0.8187 - val_loss: 0.3216 - val_accuracy: 0.8042
Epoch 785/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3203 - accuracy: 0.8097 - val_loss: 0.3296 - val_accuracy: 0.8252
Epoch 786/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3116 - accuracy: 0.8157 - val_loss: 0.3087 - val_accuracy: 0.8322
Epoch 787/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3111 - accuracy: 0.8157 - val_loss: 0.3597 - val_accuracy: 0.7832
Epoch 788/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3118 - accuracy: 0.8187 - val_loss: 0.3075 - val_accuracy: 0.8322
Epoch 789/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3075 - accuracy: 0.8218 - val_loss: 0.3033 - val_accuracy: 0.8322
Epoch 790/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3125 - accuracy: 0.8127 - val_loss: 0

Epoch 840/1000
331/331 [==============================] - 0s 79us/step - loss: 0.3045 - accuracy: 0.8187 - val_loss: 0.3402 - val_accuracy: 0.7972
Epoch 841/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3142 - accuracy: 0.8127 - val_loss: 0.3024 - val_accuracy: 0.8322
Epoch 842/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3219 - accuracy: 0.8036 - val_loss: 0.3037 - val_accuracy: 0.8042
Epoch 843/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3121 - accuracy: 0.8097 - val_loss: 0.3133 - val_accuracy: 0.8322
Epoch 844/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3085 - accuracy: 0.8157 - val_loss: 0.3565 - val_accuracy: 0.7832
Epoch 845/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3114 - accuracy: 0.8157 - val_loss: 0.3511 - val_accuracy: 0.7902
Epoch 846/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3210 - accuracy: 0.8066 - val_loss: 0

Epoch 896/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3572 - accuracy: 0.7553 - val_loss: 0.3647 - val_accuracy: 0.7552
Epoch 897/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3567 - accuracy: 0.7553 - val_loss: 0.3648 - val_accuracy: 0.7552
Epoch 898/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3560 - accuracy: 0.7553 - val_loss: 0.3702 - val_accuracy: 0.7552
Epoch 899/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3552 - accuracy: 0.7553 - val_loss: 0.3671 - val_accuracy: 0.7552
Epoch 900/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3546 - accuracy: 0.7553 - val_loss: 0.3662 - val_accuracy: 0.7552
Epoch 901/1000
331/331 [==============================] - 0s 66us/step - loss: 0.3541 - accuracy: 0.7553 - val_loss: 0.3537 - val_accuracy: 0.7552
Epoch 902/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3538 - accuracy: 0.7553 - val_loss: 0

Epoch 952/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3153 - accuracy: 0.8127 - val_loss: 0.3068 - val_accuracy: 0.8252
Epoch 953/1000
331/331 [==============================] - 0s 69us/step - loss: 0.2993 - accuracy: 0.8248 - val_loss: 0.3145 - val_accuracy: 0.8322
Epoch 954/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3025 - accuracy: 0.8218 - val_loss: 0.3137 - val_accuracy: 0.8252
Epoch 955/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3003 - accuracy: 0.8248 - val_loss: 0.3613 - val_accuracy: 0.7832
Epoch 956/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3270 - accuracy: 0.8006 - val_loss: 0.3131 - val_accuracy: 0.8252
Epoch 957/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3038 - accuracy: 0.8248 - val_loss: 0.3199 - val_accuracy: 0.8042
Epoch 958/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3053 - accuracy: 0.8157 - val_loss: 0

In [36]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

143/143 [==============================] - 0s 63us/step
over-sampling test accuracy: 81.12%


In [37]:
############ Feature selection using lasso ##########

In [38]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [39]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [40]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [41]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [42]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 146
selected features: 17


In [43]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  2,   3,   5,  11,  34,  45,  59,  79,  87, 103, 104, 105, 106,
        109, 113, 144, 145]])

In [44]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTAATACATAT', 'TTTTAAATATTATAA', 'TTTATCTTTATGA',
       'TTCAAGAAGGAGA', 'TCTCCTTCTTGA', 'TACAACGTCTT', 'GGAATCTTACT',
       'CTGGATCCG', 'CAATGACTATTAT', 'ribN', 'purC', 'group_5797',
       'group_8402', 'group_3457', 'group_562', 'ST', 'CC'], dtype='<U100')

In [45]:
removed_features = X_train_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([  0,   1,   4,   6,   7,   8,   9,  10,  12,  13,
            ...
            134, 135, 136, 137, 138, 139, 140, 141, 142, 143],
           dtype='int64', length=129)

In [46]:
len(removed_features)

129

In [47]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 17) (255,) (255, 18)


In [48]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 201), (1, 169)]


In [49]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 237), (1, 237)]


In [50]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [51]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [52]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [53]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [54]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 259 samples, validate on 111 samples
Epoch 1/1000
259/259 [==============================] - 0s 423us/step - loss: 17.8769 - accuracy: 0.5483 - val_loss: 0.7706 - val_accuracy: 0.5405
Epoch 2/1000
259/259 [==============================] - 0s 74us/step - loss: 0.6989 - accuracy: 0.5367 - val_loss: 0.6702 - val_accuracy: 0.6126
Epoch 3/1000
259/259 [==============================] - 0s 93us/step - loss: 0.6624 - accuracy: 0.5676 - val_loss: 1.0409 - val_accuracy: 0.5586
Epoch 4/1000
259/259 [==============================] - 0s 78us/step - loss: 0.8127 - accuracy: 0.5753 - val_loss: 0.7069 - val_accuracy: 0.5045
Epoch 5/1000
259/259 [==============================] - 0s 85us/step - loss: 0.6682 - accuracy: 0.5792 - val_loss: 0.6526 - val_accuracy: 0.7387
Epoch 6/1000
259/259 [==============================] - 0s 83us/step - loss: 0.6682 - accuracy: 0.6602 - val_loss: 0.6456 - val_accuracy: 0.6036
Epoch 7/1000
259/259 [==============================] - 0s 83us/step - loss: 0.646

Epoch 57/1000
259/259 [==============================] - 0s 79us/step - loss: 0.5460 - accuracy: 0.7761 - val_loss: 0.5317 - val_accuracy: 0.8198
Epoch 58/1000
259/259 [==============================] - 0s 81us/step - loss: 0.5518 - accuracy: 0.7529 - val_loss: 0.5678 - val_accuracy: 0.6847
Epoch 59/1000
259/259 [==============================] - 0s 78us/step - loss: 0.5839 - accuracy: 0.7259 - val_loss: 0.5739 - val_accuracy: 0.7838
Epoch 60/1000
259/259 [==============================] - 0s 79us/step - loss: 0.5546 - accuracy: 0.7413 - val_loss: 0.5712 - val_accuracy: 0.7027
Epoch 61/1000
259/259 [==============================] - 0s 80us/step - loss: 0.5447 - accuracy: 0.7375 - val_loss: 0.5510 - val_accuracy: 0.7748
Epoch 62/1000
259/259 [==============================] - 0s 77us/step - loss: 0.5580 - accuracy: 0.7336 - val_loss: 0.9770 - val_accuracy: 0.6216
Epoch 63/1000
259/259 [==============================] - 0s 78us/step - loss: 0.6289 - accuracy: 0.7413 - val_loss: 0.5313 -

259/259 [==============================] - 0s 74us/step - loss: 0.6959 - accuracy: 0.7452 - val_loss: 0.5178 - val_accuracy: 0.7928
Epoch 114/1000
259/259 [==============================] - 0s 84us/step - loss: 0.4847 - accuracy: 0.7876 - val_loss: 1.1525 - val_accuracy: 0.6216
Epoch 115/1000
259/259 [==============================] - 0s 112us/step - loss: 0.8049 - accuracy: 0.6448 - val_loss: 0.5132 - val_accuracy: 0.8198
Epoch 116/1000
259/259 [==============================] - 0s 83us/step - loss: 0.4933 - accuracy: 0.7954 - val_loss: 0.4962 - val_accuracy: 0.8108
Epoch 117/1000
259/259 [==============================] - 0s 76us/step - loss: 0.4990 - accuracy: 0.7683 - val_loss: 1.1942 - val_accuracy: 0.5586
Epoch 118/1000
259/259 [==============================] - 0s 77us/step - loss: 0.7101 - accuracy: 0.6911 - val_loss: 0.5373 - val_accuracy: 0.7928
Epoch 119/1000
259/259 [==============================] - 0s 73us/step - loss: 0.5239 - accuracy: 0.7568 - val_loss: 0.5276 - val_ac

Epoch 169/1000
259/259 [==============================] - 0s 74us/step - loss: 0.4576 - accuracy: 0.7799 - val_loss: 0.3978 - val_accuracy: 0.8198
Epoch 170/1000
259/259 [==============================] - 0s 72us/step - loss: 0.4253 - accuracy: 0.7992 - val_loss: 0.4079 - val_accuracy: 0.8198
Epoch 171/1000
259/259 [==============================] - 0s 69us/step - loss: 0.4532 - accuracy: 0.7799 - val_loss: 0.4320 - val_accuracy: 0.8198
Epoch 172/1000
259/259 [==============================] - 0s 71us/step - loss: 0.4298 - accuracy: 0.7838 - val_loss: 0.3927 - val_accuracy: 0.8108
Epoch 173/1000
259/259 [==============================] - 0s 93us/step - loss: 0.4177 - accuracy: 0.7992 - val_loss: 0.4052 - val_accuracy: 0.8198
Epoch 174/1000
259/259 [==============================] - 0s 73us/step - loss: 0.4372 - accuracy: 0.7838 - val_loss: 0.4622 - val_accuracy: 0.8108
Epoch 175/1000
259/259 [==============================] - 0s 99us/step - loss: 0.4744 - accuracy: 0.7761 - val_loss: 0

Epoch 225/1000
259/259 [==============================] - 0s 72us/step - loss: 0.3618 - accuracy: 0.8069 - val_loss: 0.3699 - val_accuracy: 0.8378
Epoch 226/1000
259/259 [==============================] - 0s 73us/step - loss: 0.4275 - accuracy: 0.7876 - val_loss: 0.3479 - val_accuracy: 0.8198
Epoch 227/1000
259/259 [==============================] - 0s 74us/step - loss: 0.3995 - accuracy: 0.8031 - val_loss: 0.3487 - val_accuracy: 0.8288
Epoch 228/1000
259/259 [==============================] - 0s 73us/step - loss: 0.4895 - accuracy: 0.7529 - val_loss: 0.6442 - val_accuracy: 0.8018
Epoch 229/1000
259/259 [==============================] - 0s 73us/step - loss: 0.4390 - accuracy: 0.7876 - val_loss: 0.5365 - val_accuracy: 0.7027
Epoch 230/1000
259/259 [==============================] - 0s 75us/step - loss: 0.5066 - accuracy: 0.7645 - val_loss: 0.3473 - val_accuracy: 0.8288
Epoch 231/1000
259/259 [==============================] - 0s 72us/step - loss: 0.3675 - accuracy: 0.8031 - val_loss: 0

...

259/259 [==============================] - 0s 76us/step - loss: 0.3217 - accuracy: 0.8378 - val_loss: 1.1188 - val_accuracy: 0.7387
Epoch 727/1000
259/259 [==============================] - 0s 76us/step - loss: 0.4305 - accuracy: 0.8301 - val_loss: 0.2977 - val_accuracy: 0.8829
Epoch 728/1000
259/259 [==============================] - 0s 68us/step - loss: 0.2767 - accuracy: 0.8533 - val_loss: 0.2941 - val_accuracy: 0.8919
Epoch 729/1000
259/259 [==============================] - 0s 81us/step - loss: 0.2738 - accuracy: 0.8649 - val_loss: 0.3154 - val_accuracy: 0.8649
Epoch 730/1000
259/259 [==============================] - 0s 80us/step - loss: 0.2784 - accuracy: 0.8417 - val_loss: 0.4988 - val_accuracy: 0.8559
Epoch 731/1000
259/259 [==============================] - 0s 75us/step - loss: 0.3646 - accuracy: 0.8378 - val_loss: 0.2957 - val_accuracy: 0.8559
Epoch 732/1000
259/259 [==============================] - 0s 85us/step - loss: 0.2757 - accuracy: 0.8417 - val_loss: 0.2907 - val_acc

Epoch 782/1000
259/259 [==============================] - 0s 82us/step - loss: 0.3521 - accuracy: 0.8224 - val_loss: 0.3264 - val_accuracy: 0.8919
Epoch 783/1000
259/259 [==============================] - 0s 71us/step - loss: 0.3160 - accuracy: 0.8456 - val_loss: 0.3383 - val_accuracy: 0.8649
Epoch 784/1000
259/259 [==============================] - 0s 76us/step - loss: 0.3022 - accuracy: 0.8417 - val_loss: 0.3022 - val_accuracy: 0.8739
Epoch 785/1000
259/259 [==============================] - 0s 82us/step - loss: 0.3017 - accuracy: 0.8494 - val_loss: 0.3642 - val_accuracy: 0.8649
Epoch 786/1000
259/259 [==============================] - 0s 75us/step - loss: 0.3095 - accuracy: 0.8417 - val_loss: 0.3038 - val_accuracy: 0.8739
Epoch 787/1000
259/259 [==============================] - 0s 77us/step - loss: 0.2990 - accuracy: 0.8417 - val_loss: 0.3205 - val_accuracy: 0.9009
Epoch 788/1000
259/259 [==============================] - 0s 69us/step - loss: 0.2981 - accuracy: 0.8494 - val_loss: 0

259/259 [==============================] - 0s 79us/step - loss: 0.2819 - accuracy: 0.8340 - val_loss: 0.4585 - val_accuracy: 0.8559
Epoch 838/1000
259/259 [==============================] - 0s 87us/step - loss: 0.3419 - accuracy: 0.8378 - val_loss: 0.2881 - val_accuracy: 0.8559
Epoch 839/1000
259/259 [==============================] - 0s 83us/step - loss: 0.2799 - accuracy: 0.8456 - val_loss: 0.2827 - val_accuracy: 0.8559
Epoch 840/1000
259/259 [==============================] - 0s 125us/step - loss: 0.2847 - accuracy: 0.8340 - val_loss: 0.2849 - val_accuracy: 0.8919
Epoch 841/1000
259/259 [==============================] - 0s 162us/step - loss: 0.2704 - accuracy: 0.8533 - val_loss: 0.2936 - val_accuracy: 0.8559
Epoch 842/1000
259/259 [==============================] - 0s 141us/step - loss: 0.3374 - accuracy: 0.8185 - val_loss: 0.2963 - val_accuracy: 0.8739
Epoch 843/1000
259/259 [==============================] - 0s 145us/step - loss: 0.2932 - accuracy: 0.8417 - val_loss: 0.2882 - val

Epoch 893/1000
259/259 [==============================] - 0s 187us/step - loss: 0.2536 - accuracy: 0.8571 - val_loss: 0.2743 - val_accuracy: 0.8919
Epoch 894/1000
259/259 [==============================] - 0s 148us/step - loss: 0.2548 - accuracy: 0.8649 - val_loss: 0.2729 - val_accuracy: 0.8559
Epoch 895/1000
259/259 [==============================] - 0s 88us/step - loss: 0.2552 - accuracy: 0.8610 - val_loss: 0.2669 - val_accuracy: 0.8649
Epoch 896/1000
259/259 [==============================] - 0s 98us/step - loss: 0.2509 - accuracy: 0.8610 - val_loss: 0.2703 - val_accuracy: 0.8649
Epoch 897/1000
259/259 [==============================] - 0s 73us/step - loss: 0.2555 - accuracy: 0.8533 - val_loss: 0.2705 - val_accuracy: 0.8559
Epoch 898/1000
259/259 [==============================] - 0s 96us/step - loss: 0.2536 - accuracy: 0.8610 - val_loss: 0.3151 - val_accuracy: 0.8919
Epoch 899/1000
259/259 [==============================] - 0s 97us/step - loss: 0.2814 - accuracy: 0.8417 - val_loss:

259/259 [==============================] - 0s 108us/step - loss: 0.4912 - accuracy: 0.8417 - val_loss: 0.3753 - val_accuracy: 0.8559
Epoch 949/1000
259/259 [==============================] - 0s 80us/step - loss: 0.3276 - accuracy: 0.8417 - val_loss: 0.3188 - val_accuracy: 0.8739
Epoch 950/1000
259/259 [==============================] - 0s 83us/step - loss: 0.2868 - accuracy: 0.8494 - val_loss: 0.2899 - val_accuracy: 0.8739
Epoch 951/1000
259/259 [==============================] - 0s 78us/step - loss: 0.2671 - accuracy: 0.8456 - val_loss: 0.2873 - val_accuracy: 0.8919
Epoch 952/1000
259/259 [==============================] - 0s 80us/step - loss: 0.2658 - accuracy: 0.8649 - val_loss: 0.2831 - val_accuracy: 0.8829
Epoch 953/1000
259/259 [==============================] - 0s 80us/step - loss: 0.2678 - accuracy: 0.8533 - val_loss: 0.2768 - val_accuracy: 0.8919
Epoch 954/1000
259/259 [==============================] - 0s 72us/step - loss: 0.2648 - accuracy: 0.8610 - val_loss: 0.3772 - val_ac

In [55]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


111/111 [==============================] - 0s 51us/step
combination test accuracy: 89.19%


In [56]:
#### improve neural network on combination data
model2_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [57]:
model2_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [58]:
model2_comb2.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 259 samples, validate on 111 samples
Epoch 1/1000
259/259 [==============================] - 0s 459us/step - loss: 11.3735 - accuracy: 0.5251 - val_loss: 8.8381 - val_accuracy: 0.5135
Epoch 2/1000
259/259 [==============================] - 0s 55us/step - loss: 6.2443 - accuracy: 0.5058 - val_loss: 2.6248 - val_accuracy: 0.6216
Epoch 3/1000
259/259 [==============================] - 0s 57us/step - loss: 4.7114 - accuracy: 0.5907 - val_loss: 0.7269 - val_accuracy: 0.5315
Epoch 4/1000
259/259 [==============================] - 0s 61us/step - loss: 1.4284 - accuracy: 0.5405 - val_loss: 0.6948 - val_accuracy: 0.5495
Epoch 5/1000
259/259 [==============================] - 0s 65us/step - loss: 0.7962 - accuracy: 0.5135 - val_loss: 0.8649 - val_accuracy: 0.5405
Epoch 6/1000
259/259 [==============================] - 0s 62us/step - loss: 0.8198 - accuracy: 0.5328 - val_loss: 0.8902 - val_accuracy: 0.4775
Epoch 7/1000
259/259 [==============================] - 0s 60us/step - loss: 0.956

Epoch 57/1000
259/259 [==============================] - 0s 75us/step - loss: 0.6538 - accuracy: 0.6178 - val_loss: 0.7011 - val_accuracy: 0.6937
Epoch 58/1000
259/259 [==============================] - 0s 66us/step - loss: 0.7106 - accuracy: 0.6062 - val_loss: 0.6590 - val_accuracy: 0.6486
Epoch 59/1000
259/259 [==============================] - 0s 56us/step - loss: 0.6521 - accuracy: 0.6448 - val_loss: 0.6517 - val_accuracy: 0.6937
Epoch 60/1000
259/259 [==============================] - 0s 62us/step - loss: 0.6435 - accuracy: 0.6795 - val_loss: 0.6508 - val_accuracy: 0.6667
Epoch 61/1000
259/259 [==============================] - 0s 58us/step - loss: 0.6418 - accuracy: 0.6564 - val_loss: 0.6487 - val_accuracy: 0.6847
Epoch 62/1000
259/259 [==============================] - 0s 55us/step - loss: 0.6398 - accuracy: 0.6602 - val_loss: 0.6493 - val_accuracy: 0.6757
Epoch 63/1000
259/259 [==============================] - 0s 50us/step - loss: 0.6410 - accuracy: 0.6564 - val_loss: 0.6558 -

259/259 [==============================] - 0s 55us/step - loss: 0.6014 - accuracy: 0.6988 - val_loss: 0.6060 - val_accuracy: 0.7748
Epoch 114/1000
259/259 [==============================] - 0s 58us/step - loss: 0.5956 - accuracy: 0.7259 - val_loss: 0.6043 - val_accuracy: 0.7748
Epoch 115/1000
259/259 [==============================] - 0s 57us/step - loss: 0.5941 - accuracy: 0.7336 - val_loss: 0.6055 - val_accuracy: 0.7207
Epoch 116/1000
259/259 [==============================] - 0s 51us/step - loss: 0.5951 - accuracy: 0.7336 - val_loss: 0.6044 - val_accuracy: 0.7297
Epoch 117/1000
259/259 [==============================] - 0s 62us/step - loss: 0.5952 - accuracy: 0.7529 - val_loss: 0.6155 - val_accuracy: 0.6937
Epoch 118/1000
259/259 [==============================] - 0s 54us/step - loss: 0.5955 - accuracy: 0.7413 - val_loss: 0.5993 - val_accuracy: 0.7748
Epoch 119/1000
259/259 [==============================] - 0s 59us/step - loss: 0.5902 - accuracy: 0.7297 - val_loss: 0.5981 - val_acc

Epoch 169/1000
259/259 [==============================] - 0s 60us/step - loss: 0.5494 - accuracy: 0.8031 - val_loss: 0.5506 - val_accuracy: 0.7838
Epoch 170/1000
259/259 [==============================] - 0s 50us/step - loss: 0.5420 - accuracy: 0.7799 - val_loss: 0.5465 - val_accuracy: 0.8559
Epoch 171/1000
259/259 [==============================] - 0s 61us/step - loss: 0.5420 - accuracy: 0.7761 - val_loss: 0.5445 - val_accuracy: 0.8559
Epoch 172/1000
259/259 [==============================] - 0s 54us/step - loss: 0.5374 - accuracy: 0.8031 - val_loss: 0.5438 - val_accuracy: 0.7838
Epoch 173/1000
259/259 [==============================] - 0s 58us/step - loss: 0.5369 - accuracy: 0.7876 - val_loss: 0.7583 - val_accuracy: 0.5946
Epoch 174/1000
259/259 [==============================] - 0s 52us/step - loss: 0.6286 - accuracy: 0.6525 - val_loss: 0.5430 - val_accuracy: 0.8288
Epoch 175/1000
259/259 [==============================] - 0s 65us/step - loss: 0.5405 - accuracy: 0.7876 - val_loss: 0

259/259 [==============================] - 0s 50us/step - loss: 0.8669 - accuracy: 0.6564 - val_loss: 0.5851 - val_accuracy: 0.7748
Epoch 225/1000
259/259 [==============================] - 0s 49us/step - loss: 0.5236 - accuracy: 0.7761 - val_loss: 0.5027 - val_accuracy: 0.8378
Epoch 226/1000
259/259 [==============================] - 0s 44us/step - loss: 0.4858 - accuracy: 0.8147 - val_loss: 0.4922 - val_accuracy: 0.8378
Epoch 227/1000
259/259 [==============================] - 0s 54us/step - loss: 0.4819 - accuracy: 0.8108 - val_loss: 0.4884 - val_accuracy: 0.8468
Epoch 228/1000
259/259 [==============================] - 0s 45us/step - loss: 0.4802 - accuracy: 0.8069 - val_loss: 0.4887 - val_accuracy: 0.8198
Epoch 229/1000
259/259 [==============================] - 0s 55us/step - loss: 0.4885 - accuracy: 0.7876 - val_loss: 0.5001 - val_accuracy: 0.7928
Epoch 230/1000
259/259 [==============================] - 0s 64us/step - loss: 0.4831 - accuracy: 0.8108 - val_loss: 0.4840 - val_acc

...

Epoch 728/1000
259/259 [==============================] - 0s 64us/step - loss: 0.2897 - accuracy: 0.8456 - val_loss: 0.3265 - val_accuracy: 0.8468
Epoch 729/1000
259/259 [==============================] - 0s 57us/step - loss: 0.2883 - accuracy: 0.8533 - val_loss: 0.3082 - val_accuracy: 0.8559
Epoch 730/1000
259/259 [==============================] - 0s 62us/step - loss: 0.2788 - accuracy: 0.8494 - val_loss: 0.2748 - val_accuracy: 0.8649
Epoch 731/1000
259/259 [==============================] - 0s 60us/step - loss: 0.2636 - accuracy: 0.8456 - val_loss: 1.4483 - val_accuracy: 0.6847
Epoch 732/1000
259/259 [==============================] - 0s 59us/step - loss: 0.7469 - accuracy: 0.7181 - val_loss: 0.3317 - val_accuracy: 0.8288
Epoch 733/1000
259/259 [==============================] - 0s 56us/step - loss: 0.3216 - accuracy: 0.8378 - val_loss: 0.3022 - val_accuracy: 0.8378
Epoch 734/1000
259/259 [==============================] - 0s 51us/step - loss: 0.3033 - accuracy: 0.8378 - val_loss: 0

Epoch 784/1000
259/259 [==============================] - 0s 54us/step - loss: 0.2620 - accuracy: 0.8417 - val_loss: 0.2811 - val_accuracy: 0.8919
Epoch 785/1000
259/259 [==============================] - 0s 64us/step - loss: 0.2744 - accuracy: 0.8494 - val_loss: 0.2766 - val_accuracy: 0.8649
Epoch 786/1000
259/259 [==============================] - 0s 57us/step - loss: 0.2660 - accuracy: 0.8533 - val_loss: 0.2681 - val_accuracy: 0.8468
Epoch 787/1000
259/259 [==============================] - 0s 56us/step - loss: 0.2557 - accuracy: 0.8533 - val_loss: 0.2657 - val_accuracy: 0.8468
Epoch 788/1000
259/259 [==============================] - 0s 61us/step - loss: 0.2524 - accuracy: 0.8571 - val_loss: 0.2619 - val_accuracy: 0.8559
Epoch 789/1000
259/259 [==============================] - 0s 56us/step - loss: 0.2611 - accuracy: 0.8571 - val_loss: 0.3457 - val_accuracy: 0.8468
Epoch 790/1000
259/259 [==============================] - 0s 57us/step - loss: 0.3184 - accuracy: 0.8301 - val_loss: 0

Epoch 840/1000
259/259 [==============================] - 0s 54us/step - loss: 0.2499 - accuracy: 0.8610 - val_loss: 0.2754 - val_accuracy: 0.8288
Epoch 841/1000
259/259 [==============================] - 0s 53us/step - loss: 0.2516 - accuracy: 0.8494 - val_loss: 0.3839 - val_accuracy: 0.7387
Epoch 842/1000
259/259 [==============================] - 0s 55us/step - loss: 0.3053 - accuracy: 0.7992 - val_loss: 0.2733 - val_accuracy: 0.8559
Epoch 843/1000
259/259 [==============================] - 0s 52us/step - loss: 0.2653 - accuracy: 0.8533 - val_loss: 0.2688 - val_accuracy: 0.8649
Epoch 844/1000
259/259 [==============================] - 0s 61us/step - loss: 0.2457 - accuracy: 0.8764 - val_loss: 0.2794 - val_accuracy: 0.8468
Epoch 845/1000
259/259 [==============================] - 0s 58us/step - loss: 0.2518 - accuracy: 0.8610 - val_loss: 0.3324 - val_accuracy: 0.8378
Epoch 846/1000
259/259 [==============================] - 0s 57us/step - loss: 0.2658 - accuracy: 0.8610 - val_loss: 0

Epoch 896/1000
259/259 [==============================] - 0s 61us/step - loss: 0.3081 - accuracy: 0.8456 - val_loss: 0.2893 - val_accuracy: 0.8829
Epoch 897/1000
259/259 [==============================] - 0s 61us/step - loss: 0.2974 - accuracy: 0.8533 - val_loss: 0.2830 - val_accuracy: 0.8829
Epoch 898/1000
259/259 [==============================] - 0s 52us/step - loss: 0.2915 - accuracy: 0.8533 - val_loss: 0.2794 - val_accuracy: 0.8919
Epoch 899/1000
259/259 [==============================] - 0s 52us/step - loss: 0.2860 - accuracy: 0.8494 - val_loss: 0.2786 - val_accuracy: 0.8649
Epoch 900/1000
259/259 [==============================] - 0s 65us/step - loss: 0.2806 - accuracy: 0.8378 - val_loss: 0.2769 - val_accuracy: 0.8559
Epoch 901/1000
259/259 [==============================] - 0s 55us/step - loss: 0.2706 - accuracy: 0.8378 - val_loss: 0.3593 - val_accuracy: 0.7658
Epoch 902/1000
259/259 [==============================] - 0s 57us/step - loss: 0.3084 - accuracy: 0.8224 - val_loss: 0

Epoch 952/1000
259/259 [==============================] - 0s 60us/step - loss: 0.2816 - accuracy: 0.8417 - val_loss: 0.2661 - val_accuracy: 0.8829
Epoch 953/1000
259/259 [==============================] - 0s 61us/step - loss: 0.2800 - accuracy: 0.8417 - val_loss: 0.3090 - val_accuracy: 0.7928
Epoch 954/1000
259/259 [==============================] - 0s 56us/step - loss: 0.2857 - accuracy: 0.8340 - val_loss: 0.3539 - val_accuracy: 0.8198
Epoch 955/1000
259/259 [==============================] - 0s 57us/step - loss: 0.3090 - accuracy: 0.8301 - val_loss: 0.4129 - val_accuracy: 0.8018
Epoch 956/1000
259/259 [==============================] - 0s 51us/step - loss: 0.3628 - accuracy: 0.7761 - val_loss: 0.2885 - val_accuracy: 0.8468
Epoch 957/1000
259/259 [==============================] - 0s 53us/step - loss: 0.2979 - accuracy: 0.8378 - val_loss: 0.2807 - val_accuracy: 0.8559
Epoch 958/1000
259/259 [==============================] - 0s 53us/step - loss: 0.2898 - accuracy: 0.8494 - val_loss: 0

In [59]:
acc_test_comb3 = model2_comb2.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

111/111 [==============================] - 0s 79us/step
combination test accuracy: 85.59%


In [60]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [61]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [62]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 353us/step - loss: 9.0534 - accuracy: 0.4230 - val_loss: 0.7160 - val_accuracy: 0.4336
Epoch 2/1000
331/331 [==============================] - 0s 65us/step - loss: 0.7086 - accuracy: 0.4441 - val_loss: 0.7138 - val_accuracy: 0.4895
Epoch 3/1000
331/331 [==============================] - 0s 70us/step - loss: 0.6984 - accuracy: 0.5106 - val_loss: 0.7168 - val_accuracy: 0.4196
Epoch 4/1000
331/331 [==============================] - 0s 70us/step - loss: 0.6984 - accuracy: 0.5045 - val_loss: 0.7015 - val_accuracy: 0.4895
Epoch 5/1000
331/331 [==============================] - 0s 73us/step - loss: 0.6936 - accuracy: 0.5196 - val_loss: 0.6971 - val_accuracy: 0.4266
Epoch 6/1000
331/331 [==============================] - 0s 65us/step - loss: 0.6914 - accuracy: 0.5166 - val_loss: 0.7013 - val_accuracy: 0.3916
Epoch 7/1000
331/331 [==============================] - 0s 72us/step - loss: 0.6912

Epoch 57/1000
331/331 [==============================] - 0s 71us/step - loss: 0.6501 - accuracy: 0.6888 - val_loss: 0.6564 - val_accuracy: 0.6853
Epoch 58/1000
331/331 [==============================] - 0s 65us/step - loss: 0.6438 - accuracy: 0.7069 - val_loss: 0.6545 - val_accuracy: 0.6993
Epoch 59/1000
331/331 [==============================] - 0s 63us/step - loss: 0.6416 - accuracy: 0.7190 - val_loss: 0.6542 - val_accuracy: 0.6154
Epoch 60/1000
331/331 [==============================] - 0s 63us/step - loss: 0.6711 - accuracy: 0.6314 - val_loss: 0.6777 - val_accuracy: 0.5455
Epoch 61/1000
331/331 [==============================] - 0s 62us/step - loss: 0.6541 - accuracy: 0.6073 - val_loss: 0.6768 - val_accuracy: 0.5455
Epoch 62/1000
331/331 [==============================] - 0s 74us/step - loss: 0.6513 - accuracy: 0.6163 - val_loss: 0.6741 - val_accuracy: 0.5944
Epoch 63/1000
331/331 [==============================] - 0s 67us/step - loss: 0.6496 - accuracy: 0.6526 - val_loss: 0.6796 -

331/331 [==============================] - 0s 62us/step - loss: 0.6022 - accuracy: 0.6949 - val_loss: 0.6706 - val_accuracy: 0.5664
Epoch 114/1000
331/331 [==============================] - 0s 64us/step - loss: 0.6211 - accuracy: 0.6616 - val_loss: 0.6193 - val_accuracy: 0.6224
Epoch 115/1000
331/331 [==============================] - 0s 63us/step - loss: 0.6008 - accuracy: 0.6979 - val_loss: 0.6064 - val_accuracy: 0.6923
Epoch 116/1000
331/331 [==============================] - 0s 76us/step - loss: 0.5976 - accuracy: 0.7251 - val_loss: 0.6068 - val_accuracy: 0.6503
Epoch 117/1000
331/331 [==============================] - 0s 64us/step - loss: 0.5969 - accuracy: 0.7100 - val_loss: 0.6026 - val_accuracy: 0.6783
Epoch 118/1000
331/331 [==============================] - 0s 59us/step - loss: 0.5961 - accuracy: 0.7130 - val_loss: 0.6033 - val_accuracy: 0.6503
Epoch 119/1000
331/331 [==============================] - 0s 65us/step - loss: 0.5944 - accuracy: 0.7311 - val_loss: 0.6026 - val_acc

Epoch 169/1000
331/331 [==============================] - 0s 64us/step - loss: 0.6100 - accuracy: 0.6133 - val_loss: 0.6089 - val_accuracy: 0.7413
Epoch 170/1000
331/331 [==============================] - 0s 66us/step - loss: 0.5982 - accuracy: 0.6858 - val_loss: 0.6072 - val_accuracy: 0.7552
Epoch 171/1000
331/331 [==============================] - 0s 65us/step - loss: 0.5965 - accuracy: 0.7281 - val_loss: 0.6064 - val_accuracy: 0.7552
Epoch 172/1000
331/331 [==============================] - 0s 64us/step - loss: 0.5955 - accuracy: 0.7190 - val_loss: 0.6074 - val_accuracy: 0.7552
Epoch 173/1000
331/331 [==============================] - 0s 63us/step - loss: 0.5991 - accuracy: 0.7251 - val_loss: 0.6075 - val_accuracy: 0.7692
Epoch 174/1000
331/331 [==============================] - 0s 67us/step - loss: 0.5981 - accuracy: 0.7402 - val_loss: 0.6081 - val_accuracy: 0.7483
Epoch 175/1000
331/331 [==============================] - 0s 65us/step - loss: 0.5959 - accuracy: 0.7341 - val_loss: 0

Epoch 225/1000
331/331 [==============================] - 0s 64us/step - loss: 0.5577 - accuracy: 0.7402 - val_loss: 0.5570 - val_accuracy: 0.7413
Epoch 226/1000
331/331 [==============================] - 0s 58us/step - loss: 0.5521 - accuracy: 0.7613 - val_loss: 0.5649 - val_accuracy: 0.7413
Epoch 227/1000
331/331 [==============================] - 0s 64us/step - loss: 0.5603 - accuracy: 0.7372 - val_loss: 0.5565 - val_accuracy: 0.7832
Epoch 228/1000
331/331 [==============================] - 0s 64us/step - loss: 0.5568 - accuracy: 0.7462 - val_loss: 0.5684 - val_accuracy: 0.7273
Epoch 229/1000
331/331 [==============================] - 0s 63us/step - loss: 0.5454 - accuracy: 0.7764 - val_loss: 0.5521 - val_accuracy: 0.7413
Epoch 230/1000
331/331 [==============================] - 0s 61us/step - loss: 0.5582 - accuracy: 0.7492 - val_loss: 0.5444 - val_accuracy: 0.7413
Epoch 231/1000
331/331 [==============================] - 0s 66us/step - loss: 0.5470 - accuracy: 0.7553 - val_loss: 0

...

Epoch 729/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4232 - accuracy: 0.7764 - val_loss: 0.4163 - val_accuracy: 0.7343
Epoch 730/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4218 - accuracy: 0.7674 - val_loss: 0.4164 - val_accuracy: 0.7692
Epoch 731/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4310 - accuracy: 0.7553 - val_loss: 0.4071 - val_accuracy: 0.7692
Epoch 732/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4216 - accuracy: 0.7795 - val_loss: 0.4390 - val_accuracy: 0.7622
Epoch 733/1000
331/331 [==============================] - 0s 62us/step - loss: 0.4190 - accuracy: 0.7734 - val_loss: 0.4831 - val_accuracy: 0.7483
Epoch 734/1000
331/331 [==============================] - 0s 63us/step - loss: 0.4240 - accuracy: 0.7704 - val_loss: 0.5580 - val_accuracy: 0.7273
Epoch 735/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4686 - accuracy: 0.7462 - val_loss: 0

Epoch 785/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4387 - accuracy: 0.7704 - val_loss: 0.4063 - val_accuracy: 0.7692
Epoch 786/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4310 - accuracy: 0.7613 - val_loss: 0.4169 - val_accuracy: 0.7692
Epoch 787/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4357 - accuracy: 0.7553 - val_loss: 0.4163 - val_accuracy: 0.7692
Epoch 788/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4296 - accuracy: 0.7704 - val_loss: 0.4139 - val_accuracy: 0.7692
Epoch 789/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4260 - accuracy: 0.7583 - val_loss: 0.4085 - val_accuracy: 0.7692
Epoch 790/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4280 - accuracy: 0.7674 - val_loss: 0.4139 - val_accuracy: 0.7692
Epoch 791/1000
331/331 [==============================] - 0s 63us/step - loss: 0.4283 - accuracy: 0.7704 - val_loss: 0

Epoch 841/1000
331/331 [==============================] - 0s 69us/step - loss: 0.4251 - accuracy: 0.7704 - val_loss: 0.4106 - val_accuracy: 0.7692
Epoch 842/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4238 - accuracy: 0.7674 - val_loss: 0.4092 - val_accuracy: 0.7692
Epoch 843/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4236 - accuracy: 0.7704 - val_loss: 0.4017 - val_accuracy: 0.7902
Epoch 844/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4227 - accuracy: 0.7704 - val_loss: 0.4108 - val_accuracy: 0.7692
Epoch 845/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4189 - accuracy: 0.7764 - val_loss: 0.4083 - val_accuracy: 0.7762
Epoch 846/1000
331/331 [==============================] - 0s 63us/step - loss: 0.4209 - accuracy: 0.7795 - val_loss: 0.4343 - val_accuracy: 0.7483
Epoch 847/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4240 - accuracy: 0.7764 - val_loss: 0

Epoch 897/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4217 - accuracy: 0.7613 - val_loss: 0.4068 - val_accuracy: 0.7692
Epoch 898/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4229 - accuracy: 0.7613 - val_loss: 0.4077 - val_accuracy: 0.7692
Epoch 899/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4218 - accuracy: 0.7704 - val_loss: 0.4069 - val_accuracy: 0.7692
Epoch 900/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4192 - accuracy: 0.7704 - val_loss: 0.4076 - val_accuracy: 0.7692
Epoch 901/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4186 - accuracy: 0.7704 - val_loss: 0.4048 - val_accuracy: 0.7692
Epoch 902/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4198 - accuracy: 0.7674 - val_loss: 0.3996 - val_accuracy: 0.7692
Epoch 903/1000
331/331 [==============================] - 0s 63us/step - loss: 0.4149 - accuracy: 0.7764 - val_loss: 0

Epoch 953/1000
331/331 [==============================] - 0s 68us/step - loss: 0.4207 - accuracy: 0.7764 - val_loss: 0.4039 - val_accuracy: 0.7692
Epoch 954/1000
331/331 [==============================] - 0s 76us/step - loss: 0.4168 - accuracy: 0.7734 - val_loss: 0.4037 - val_accuracy: 0.7692
Epoch 955/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4113 - accuracy: 0.7795 - val_loss: 0.3956 - val_accuracy: 0.7902
Epoch 956/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4158 - accuracy: 0.7704 - val_loss: 0.4037 - val_accuracy: 0.7692
Epoch 957/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4090 - accuracy: 0.7795 - val_loss: 0.3932 - val_accuracy: 0.7902
Epoch 958/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4103 - accuracy: 0.7764 - val_loss: 0.4044 - val_accuracy: 0.7692
Epoch 959/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4098 - accuracy: 0.7734 - val_loss: 0

In [63]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

143/143 [==============================] - 0s 64us/step
combination test accuracy: 79.02%


In [64]:
#### improve neural network on over-sampling data
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [65]:
model2_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [66]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 383us/step - loss: 7.8444 - accuracy: 0.5076 - val_loss: 0.6930 - val_accuracy: 0.5804
Epoch 2/1000
331/331 [==============================] - 0s 67us/step - loss: 0.7506 - accuracy: 0.5378 - val_loss: 0.7059 - val_accuracy: 0.4755
Epoch 3/1000
331/331 [==============================] - 0s 76us/step - loss: 0.7385 - accuracy: 0.5347 - val_loss: 0.8095 - val_accuracy: 0.5315
Epoch 4/1000
331/331 [==============================] - 0s 73us/step - loss: 0.7058 - accuracy: 0.5801 - val_loss: 0.7334 - val_accuracy: 0.5664
Epoch 5/1000
331/331 [==============================] - 0s 70us/step - loss: 0.6978 - accuracy: 0.5831 - val_loss: 0.6906 - val_accuracy: 0.6154
Epoch 6/1000
331/331 [==============================] - 0s 74us/step - loss: 0.7388 - accuracy: 0.5589 - val_loss: 0.6826 - val_accuracy: 0.6643
Epoch 7/1000
331/331 [==============================] - 0s 70us/step - loss: 0.6859

Epoch 57/1000
331/331 [==============================] - 0s 67us/step - loss: 0.6456 - accuracy: 0.6707 - val_loss: 0.6356 - val_accuracy: 0.7692
Epoch 58/1000
331/331 [==============================] - 0s 65us/step - loss: 0.6452 - accuracy: 0.6918 - val_loss: 0.6494 - val_accuracy: 0.6853
Epoch 59/1000
331/331 [==============================] - 0s 69us/step - loss: 0.6389 - accuracy: 0.7130 - val_loss: 0.6431 - val_accuracy: 0.7133
Epoch 60/1000
331/331 [==============================] - 0s 65us/step - loss: 0.6429 - accuracy: 0.6616 - val_loss: 0.6318 - val_accuracy: 0.6853
Epoch 61/1000
331/331 [==============================] - 0s 62us/step - loss: 0.6387 - accuracy: 0.6949 - val_loss: 0.6361 - val_accuracy: 0.7273
Epoch 62/1000
331/331 [==============================] - 0s 66us/step - loss: 0.6387 - accuracy: 0.7069 - val_loss: 0.6428 - val_accuracy: 0.7133
Epoch 63/1000
331/331 [==============================] - 0s 64us/step - loss: 0.6396 - accuracy: 0.6979 - val_loss: 0.6880 -

331/331 [==============================] - 0s 81us/step - loss: 0.5853 - accuracy: 0.7039 - val_loss: 0.5625 - val_accuracy: 0.7413
Epoch 114/1000
331/331 [==============================] - 0s 76us/step - loss: 0.5946 - accuracy: 0.7100 - val_loss: 0.5497 - val_accuracy: 0.7622
Epoch 115/1000
331/331 [==============================] - 0s 81us/step - loss: 0.5663 - accuracy: 0.7372 - val_loss: 0.5745 - val_accuracy: 0.7273
Epoch 116/1000
331/331 [==============================] - 0s 70us/step - loss: 0.5745 - accuracy: 0.7311 - val_loss: 0.5733 - val_accuracy: 0.7133
Epoch 117/1000
331/331 [==============================] - 0s 79us/step - loss: 0.5776 - accuracy: 0.7160 - val_loss: 0.6095 - val_accuracy: 0.7273
Epoch 118/1000
331/331 [==============================] - 0s 75us/step - loss: 0.5707 - accuracy: 0.7009 - val_loss: 0.5939 - val_accuracy: 0.7133
Epoch 119/1000
331/331 [==============================] - 0s 74us/step - loss: 0.5796 - accuracy: 0.7160 - val_loss: 0.5439 - val_acc

Epoch 169/1000
331/331 [==============================] - 0s 76us/step - loss: 0.5381 - accuracy: 0.7402 - val_loss: 0.4724 - val_accuracy: 0.8042
Epoch 170/1000
331/331 [==============================] - 0s 74us/step - loss: 0.5242 - accuracy: 0.7583 - val_loss: 0.5089 - val_accuracy: 0.7483
Epoch 171/1000
331/331 [==============================] - 0s 69us/step - loss: 0.5233 - accuracy: 0.7372 - val_loss: 0.5104 - val_accuracy: 0.7273
Epoch 172/1000
331/331 [==============================] - 0s 71us/step - loss: 0.5130 - accuracy: 0.7523 - val_loss: 0.4882 - val_accuracy: 0.7692
Epoch 173/1000
331/331 [==============================] - 0s 72us/step - loss: 0.5078 - accuracy: 0.7462 - val_loss: 0.4834 - val_accuracy: 0.7133
Epoch 174/1000
331/331 [==============================] - 0s 69us/step - loss: 0.4996 - accuracy: 0.7462 - val_loss: 0.4693 - val_accuracy: 0.7692
Epoch 175/1000
331/331 [==============================] - 0s 77us/step - loss: 0.4971 - accuracy: 0.7583 - val_loss: 0

Epoch 225/1000
331/331 [==============================] - 0s 72us/step - loss: 0.4464 - accuracy: 0.7825 - val_loss: 0.4278 - val_accuracy: 0.7692
Epoch 226/1000
331/331 [==============================] - 0s 72us/step - loss: 0.4507 - accuracy: 0.7674 - val_loss: 0.5090 - val_accuracy: 0.7133
Epoch 227/1000
331/331 [==============================] - 0s 72us/step - loss: 0.4535 - accuracy: 0.7704 - val_loss: 0.4052 - val_accuracy: 0.7692
Epoch 228/1000
331/331 [==============================] - 0s 70us/step - loss: 0.4588 - accuracy: 0.7553 - val_loss: 0.4247 - val_accuracy: 0.7483
Epoch 229/1000
331/331 [==============================] - 0s 80us/step - loss: 0.4645 - accuracy: 0.7583 - val_loss: 0.4428 - val_accuracy: 0.7273
Epoch 230/1000
331/331 [==============================] - 0s 69us/step - loss: 0.4572 - accuracy: 0.7613 - val_loss: 0.4221 - val_accuracy: 0.7692
Epoch 231/1000
331/331 [==============================] - 0s 73us/step - loss: 0.4394 - accuracy: 0.7825 - val_loss: 0

...

331/331 [==============================] - 0s 71us/step - loss: 0.3411 - accuracy: 0.8066 - val_loss: 0.3382 - val_accuracy: 0.8112
Epoch 727/1000
331/331 [==============================] - 0s 76us/step - loss: 0.3805 - accuracy: 0.7885 - val_loss: 0.3611 - val_accuracy: 0.7832
Epoch 728/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3590 - accuracy: 0.8036 - val_loss: 0.3810 - val_accuracy: 0.7832
Epoch 729/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3734 - accuracy: 0.7976 - val_loss: 0.4605 - val_accuracy: 0.7273
Epoch 730/1000
331/331 [==============================] - ETA: 0s - loss: 0.6391 - accuracy: 0.62 - 0s 74us/step - loss: 0.3874 - accuracy: 0.7734 - val_loss: 0.3796 - val_accuracy: 0.7902
Epoch 731/1000
331/331 [==============================] - 0s 74us/step - loss: 0.4051 - accuracy: 0.7704 - val_loss: 0.3830 - val_accuracy: 0.7762
Epoch 732/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3843 - acc

331/331 [==============================] - 0s 68us/step - loss: 0.3483 - accuracy: 0.7976 - val_loss: 0.3173 - val_accuracy: 0.8392
Epoch 782/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3451 - accuracy: 0.8036 - val_loss: 0.3265 - val_accuracy: 0.8252
Epoch 783/1000
331/331 [==============================] - 0s 77us/step - loss: 0.3411 - accuracy: 0.7946 - val_loss: 0.4141 - val_accuracy: 0.7552
Epoch 784/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3495 - accuracy: 0.8036 - val_loss: 0.4219 - val_accuracy: 0.7832
Epoch 785/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3408 - accuracy: 0.8066 - val_loss: 0.3691 - val_accuracy: 0.7902
Epoch 786/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3741 - accuracy: 0.7855 - val_loss: 0.3337 - val_accuracy: 0.8042
Epoch 787/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3292 - accuracy: 0.8127 - val_loss: 0.3236 - val_acc

Epoch 837/1000
331/331 [==============================] - 0s 73us/step - loss: 0.4117 - accuracy: 0.7734 - val_loss: 0.3300 - val_accuracy: 0.8182
Epoch 838/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3284 - accuracy: 0.8187 - val_loss: 0.3276 - val_accuracy: 0.8042
Epoch 839/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3499 - accuracy: 0.7946 - val_loss: 0.3783 - val_accuracy: 0.7832
Epoch 840/1000
331/331 [==============================] - 0s 66us/step - loss: 0.3427 - accuracy: 0.8097 - val_loss: 0.3932 - val_accuracy: 0.7832
Epoch 841/1000
331/331 [==============================] - 0s 65us/step - loss: 0.3351 - accuracy: 0.8097 - val_loss: 0.3627 - val_accuracy: 0.7902
Epoch 842/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3305 - accuracy: 0.8127 - val_loss: 0.4165 - val_accuracy: 0.7552
Epoch 843/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3635 - accuracy: 0.8127 - val_loss: 0

Epoch 893/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3829 - accuracy: 0.7704 - val_loss: 0.3368 - val_accuracy: 0.8042
Epoch 894/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3463 - accuracy: 0.8036 - val_loss: 0.3771 - val_accuracy: 0.7832
Epoch 895/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3772 - accuracy: 0.7764 - val_loss: 0.3151 - val_accuracy: 0.8392
Epoch 896/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3327 - accuracy: 0.8097 - val_loss: 0.3156 - val_accuracy: 0.8392
Epoch 897/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3307 - accuracy: 0.8127 - val_loss: 0.3235 - val_accuracy: 0.8042
Epoch 898/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3403 - accuracy: 0.8066 - val_loss: 0.3757 - val_accuracy: 0.7902
Epoch 899/1000
331/331 [==============================] - 0s 65us/step - loss: 0.3246 - accuracy: 0.8157 - val_loss: 0

Epoch 949/1000
331/331 [==============================] - 0s 81us/step - loss: 0.3456 - accuracy: 0.8006 - val_loss: 0.3384 - val_accuracy: 0.8042
Epoch 950/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3270 - accuracy: 0.8097 - val_loss: 0.4161 - val_accuracy: 0.7832
Epoch 951/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3555 - accuracy: 0.7946 - val_loss: 0.5503 - val_accuracy: 0.7063
Epoch 952/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3869 - accuracy: 0.7583 - val_loss: 0.3554 - val_accuracy: 0.7832
Epoch 953/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3604 - accuracy: 0.7885 - val_loss: 0.3590 - val_accuracy: 0.8112
Epoch 954/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3609 - accuracy: 0.7976 - val_loss: 0.4055 - val_accuracy: 0.7343
Epoch 955/1000
331/331 [==============================] - 0s 65us/step - loss: 0.3736 - accuracy: 0.7734 - val_loss: 0

In [67]:
acc_test_over3 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over3*100))

143/143 [==============================] - 0s 63us/step
combination test accuracy: 72.03%


In [68]:
############ Logistic Regression ############

In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [70]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [71]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 96.43%


In [72]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [73]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 94.41%


In [74]:
############## Random Forest ##############

In [75]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100,random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [76]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 96.43%


In [77]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100,random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [78]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 95.10%


In [79]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [80]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.98113208 1.         0.98076923 0.98039216 0.98039216]
0.9845371239932839


In [81]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.98507463 0.96969697 0.98484848 1.         0.93939394]
0.9758028041610132
